In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

In [3]:
#I will deal only with Comunidad de Madrid to save resources. 
# If needed, the whole set or another subset can be used, the polygons are there. 

#spain=gpd.read_file('polygons/georef-spain-municipio@public.geojson')   # cargar datos geoespaciales (multipolygon)
#RangeIndex: 8223 entries, 0 to 8222 ; Data columns (total 12 columns) ; memory usage: 6.8 MB

#madri = spain[spain['prov_name'] == 'Madrid']
# Int64Index: 181 entries, 176 to 8027 ; Data columns (total 12 columns) ; memory usage: 151.5 KB

#Only Comunidad de Madrid saved.
#madri.to_file('polygons/Comunidad_de_Madrid.geojson', driver='GeoJSON')

In [4]:
#This file contains the polygons for Comunidad de Madrid, for each Municipio.
#This file does not have municipal codes, only zipcodes. Also, the name of the Municipios does not exactly match the base table.

#I can see two possible paths. 
# Either assign zipcodes to every municipio on the base table, but some municipios have several zipcodes. 
#  Only Alaclá de Henares has 7 different zipcodes
#   mun_code is not zipcode, it is its code in the Agencia Tributaria
#    Mun_id in foreign_born is the code for Institudo de Estadística
#     Most likely one is the code for Comunidad de Madrid and the other one is in Spain
# Or clean their names in order to match the base table.
#  This will be the way.

madrid = gpd.read_file('polygons/Comunidad_de_Madrid.geojson')   # cargar datos geoespaciales (multipolygon)
#RangeIndex: 181 entries, 0 to 180 ; Data columns (total 12 columns): memory usage: 17.1+ KB

madrid.drop(madrid.columns[[1, 2, 3, 4, 5, 8, 9, 10]], axis=1, inplace=True)
#Los Baldios and El Redegüelo are not actual municipios, but territories in Cercedilla and Morarzarzal
madrid.drop(madrid[(madrid.mun_name == 'Los Baldios')].index, axis=0, inplace=True)
madrid.drop(madrid[(madrid.mun_name == 'El Redegüelo')].index, axis=0, inplace=True)
madrid.loc[madrid.mun_name == 'El Álamo', 'mun_name'] = 'El Alamo'



In [5]:
for i in madrid.mun_name: 
    if i.startswith('El '): 
        madrid.mun_name = madrid.mun_name.replace(i,i[3:] + ' (El)') 
    elif i.startswith('La '): 
        madrid.mun_name = madrid.mun_name.replace(i,i[3:] + ' (La)') 
    elif i.startswith('Las '): 
        madrid.mun_name = madrid.mun_name.replace(i,i[4:] + ' (Las)')
    elif i.startswith('Los '): 
        madrid.mun_name = madrid.mun_name.replace(i,i[4:] + ' (Los)')

madrid = madrid.sort_values('mun_name')
#madrid.columns.values[2] = 'Municipio'

In [6]:
f_b = pd.read_csv('clean_data/Foreign_born.csv')
#Columns: 212 entries, Mun_id to _merge ; dtypes: float64(1), int64(209), object(2) ; memory usage: 298.2+ KB
f_b = f_b.sort_values('Municipio')
f_b.head(10)

,Mun_id,Municipio,Poblacion_total,Porcentaje_pob_ext,Nacimiento_extranjero,Unión Europea,Resto de Europa,África,América del Norte,América Central y del Caribe,América del Sur,Otros países de América_x,Asia,Oceanía,Albania,Austria,Belgica,Bulgaria,Chipre,Dinamarca,España,Finlandia,Francia,Grecia,Hungría,Irlanda,Islandia,Italia,Liechtenstein,Luxemburgo,Malta,Mónaco,Noruega,Países Bajos,Polonia,Portugal,Andorra,Reino Unido,Alemania,Rumanía,San Marino,Santa Sede,Suecia,Suiza,Ucrania,Letonia,Moldavia,Belarús,Georgia,Estonia,Lituania,República Checa,Eslovaquia,Bosnia y Herzegovina,Croacia,Eslovenia,Armenia,Rusia,Macedonia,Serbia,Montenegro,Otros países de Europa,Burkina Faso,Angola,Argelia,Benin,Botswana,Burundi,Cabo Verde,Camerún,Comoras,Congo,Costa de Marfil,Djibouti,Egipto,Etiopía,Gabón,Gambia,Ghana,Guinea,Guinea Bissau,Guinea Ecuatorial,Kenia,Lesotho,Liberia,Libia,Madagascar,Malawi,Mali,Marruecos,Mauricio,Mauritania,Mozambique,Namibia,Níger,Nigeria,República Centroafricana,Sudáfrica,Rwanda,Santo Tomé y Príncipe,Senegal,Seychelles,Sierra Leona,Somalia,Sudán,Swazilandia,Tanzania,Chad,Togo,Túnez,Uganda,República Democrática del Congo,Zambia,Zimbabwe,Eritrea,Sudán del Sur,Otros países de África,Canadá,Estados Unidos de América,México,Antigua y Barbuda,Bahamas,Barbados,Belice,Costa Rica,Cuba,Dominica,El Salvador,Granada,Guatemala,Haití,Honduras,Jamaica,Nicaragua,Panamá,San Vicente y Las Granadinas,República Dominicana,Trinidad Y Tobago,Santa Lucía,San Cristobal y Nieves,Argentina,Bolivia,Brasil,Colombia,Chile,Ecuador,Guyana,Paraguay,Perú,Surinám,Uruguay,Venezuela,Otros países de América_y,Afganistán,Arabia Saudita,Bahrein,Bangladesh,Myanmar,China,Emiratos Árabes Unidos,Filipinas,India,Indonesia,Iraq,Irán,Israel,Japón,Jordania,Camboya,Kuwait,Laos,Líbano,Malasia,Maldivas,Mongolia,Nepal,Omán,Pakistán,Qatar,Corea del Sur,Corea del Norte,Singapur,Siria,Sri Lanka,Tailandia,Turquía,Vietnam,Brunei,Islas Marshall,Yemen,Azerbaiyán,Kazajstán,Kirguistán,Tayikistán,Turkmenistán,Uzbekistán,Bhutan,Otros países de Asia,Australia,Fiji,Nueva Zelanda,Papua Nueva Guinea,Islas Salomón,Samoa,Tonga,Vanuatu,Micronesia,Tuvalu,Timor Oriental,Apátridas,Otros países de Oceanía
0,14,Acebeda (La),55,3.636364,2,54,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,29,Ajalvir,4676,18.028229,843,4103,40,89,39,126,257,0,21,1,0,0,0,5,0,2,263,0,4,0,0,1,0,24,0,0,0,0,0,4,6,5,0,15,10,188,0,0,4,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5,2,0,0,0,0,0,0,58,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,21,5,0,0,0,0,0,49,1,7,0,0,0,0,0,4,0,0,7,0,0,0,7,5,13,24,6,7,0,3,14,0,1,40,0,0,0,0,0,0,11,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,35,Alameda del Valle,246,14.227642,35,213,1,0,2,0,30,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,6,0,0,0,0,2,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,Alamo (El),9908,19.206701,1903,8366,73,561,22,214,628,0,43,1,3,1,0,9,1,0,612,0,6,1,1,0,0,27,0,0,0,0,0,0,21,15,0,8,1,240,0,0,0,0,30,0,13,0,0,0,0,1,0,0,0,0,2,2,0,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,2,411,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,8,0,0,0,0,1,49,0,2,0,0,0,53,0,7,0,0,23,0,0,0,13,10,23,131,11,23,0,15,26,0,7,48,0,0,0,0,0,0,18,0,2,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [7]:
madrid = madrid.sort_values('mun_name')
madrid.head(10)

,geo_point_2d,mun_code,mun_name,geometry
123,"{'lon': -3.6357100678630663, 'lat': 41.0931461...",28001,Acebeda (La),"POLYGON ((-3.67606 41.08670, -3.67457 41.08504..."
0,"{'lon': -3.4801385596854346, 'lat': 40.5239757...",28002,Ajalvir,"POLYGON ((-3.51222 40.53864, -3.51151 40.53759..."
96,"{'lon': -3.8500020800172634, 'lat': 40.9203835...",28003,Alameda del Valle,"POLYGON ((-3.89864 40.95883, -3.89742 40.95681..."
164,"{'lon': -3.9892581186684386, 'lat': 40.2324046...",28004,Alamo (El),"POLYGON ((-4.02403 40.24897, -4.02443 40.24801..."
1,"{'lon': -3.3696004109155107, 'lat': 40.4945916...",28005,Alcalá de Henares,"POLYGON ((-3.44720 40.44631, -3.44669 40.44589..."
145,"{'lon': -3.6301211453858566, 'lat': 40.5345949...",28006,Alcobendas,"POLYGON ((-3.67414 40.58886, -3.67600 40.58795..."
124,"{'lon': -3.8372316857739786, 'lat': 40.3583939...",28007,Alcorcón,"POLYGON ((-3.87092 40.35312, -3.86762 40.35364..."
97,"{'lon': -4.223784484727634, 'lat': 40.30925623...",28008,Aldea del Fresno,"POLYGON ((-4.29503 40.32262, -4.29404 40.31998..."
76,"{'lon': -3.52992735290857, 'lat': 40.608218222...",28009,Algete,"POLYGON ((-3.59368 40.65480, -3.59311 40.65360..."
28,"{'lon': -4.020814002152194, 'lat': 40.66184193...",28010,Alpedrete,"POLYGON ((-4.04506 40.66253, -4.04716 40.66033..."


In [8]:
fb_geo = f_b.join(madrid)

In [9]:
fb_geo.head(2)

,Mun_id,Municipio,Poblacion_total,Porcentaje_pob_ext,Nacimiento_extranjero,Unión Europea,Resto de Europa,África,América del Norte,América Central y del Caribe,América del Sur,Otros países de América_x,Asia,Oceanía,Albania,Austria,Belgica,Bulgaria,Chipre,Dinamarca,España,Finlandia,Francia,Grecia,Hungría,Irlanda,Islandia,Italia,Liechtenstein,Luxemburgo,Malta,Mónaco,Noruega,Países Bajos,Polonia,Portugal,Andorra,Reino Unido,Alemania,Rumanía,San Marino,Santa Sede,Suecia,Suiza,Ucrania,Letonia,Moldavia,Belarús,Georgia,Estonia,Lituania,República Checa,Eslovaquia,Bosnia y Herzegovina,Croacia,Eslovenia,Armenia,Rusia,Macedonia,Serbia,Montenegro,Otros países de Europa,Burkina Faso,Angola,Argelia,Benin,Botswana,Burundi,Cabo Verde,Camerún,Comoras,Congo,Costa de Marfil,Djibouti,Egipto,Etiopía,Gabón,Gambia,Ghana,Guinea,Guinea Bissau,Guinea Ecuatorial,Kenia,Lesotho,Liberia,Libia,Madagascar,Malawi,Mali,Marruecos,Mauricio,Mauritania,Mozambique,Namibia,Níger,Nigeria,República Centroafricana,Sudáfrica,Rwanda,Santo Tomé y Príncipe,Senegal,Seychelles,Sierra Leona,Somalia,Sudán,Swazilandia,Tanzania,Chad,Togo,Túnez,Uganda,República Democrática del Congo,Zambia,Zimbabwe,Eritrea,Sudán del Sur,Otros países de África,Canadá,Estados Unidos de América,México,Antigua y Barbuda,Bahamas,Barbados,Belice,Costa Rica,Cuba,Dominica,El Salvador,Granada,Guatemala,Haití,Honduras,Jamaica,Nicaragua,Panamá,San Vicente y Las Granadinas,República Dominicana,Trinidad Y Tobago,Santa Lucía,San Cristobal y Nieves,Argentina,Bolivia,Brasil,Colombia,Chile,Ecuador,Guyana,Paraguay,Perú,Surinám,Uruguay,Venezuela,Otros países de América_y,Afganistán,Arabia Saudita,Bahrein,Bangladesh,Myanmar,China,Emiratos Árabes Unidos,Filipinas,India,Indonesia,Iraq,Irán,Israel,Japón,Jordania,Camboya,Kuwait,Laos,Líbano,Malasia,Maldivas,Mongolia,Nepal,Omán,Pakistán,Qatar,Corea del Sur,Corea del Norte,Singapur,Siria,Sri Lanka,Tailandia,Turquía,Vietnam,Brunei,Islas Marshall,Yemen,Azerbaiyán,Kazajstán,Kirguistán,Tayikistán,Turkmenistán,Uzbekistán,Bhutan,Otros países de Asia,Australia,Fiji,Nueva Zelanda,Papua Nueva Guinea,Islas Salomón,Samoa,Tonga,Vanuatu,Micronesia,Tuvalu,Timor Oriental,Apátridas,Otros países de Oceanía,geo_point_2d,mun_code,mun_name,geometry
0,14,Acebeda (La),55,3.636364,2,54,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'lon': -3.4801385596854346, 'lat': 40.5239757...",28002,Ajalvir,"POLYGON ((-3.51222 40.53864, -3.51151 40.53759..."
1,29,Ajalvir,4676,18.028229,843,4103,40,89,39,126,257,0,21,1,0,0,0,5,0,2,263,0,4,0,0,1,0,24,0,0,0,0,0,4,6,5,0,15,10,188,0,0,4,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5,2,0,0,0,0,0,0,58,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,21,5,0,0,0,0,0,49,1,7,0,0,0,0,0,4,0,0,7,0,0,0,7,5,13,24,6,7,0,3,14,0,1,40,0,0,0,0,0,0,11,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"{'lon': -3.3696004109155107, 'lat': 40.4945916...",28005,Alcalá de Henares,"POLYGON ((-3.44720 40.44631, -3.44669 40.44589..."
2,35,Alameda del Valle,246,14.227642,35,213,1,0,2,0,30,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,6,0,0,0,0,2,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'lon': -3.766947850822002, 'lat': 40.23065887...",28106,Parla,"POLYGON ((-3.80761 40.22942, -3.80746 40.22884..."
3,40,Alamo (El),9908,19.206701,1903,8366,73,5

In [10]:
#Clean data served
fb_geo.to_csv('clean_data/Geo_population.csv', index=False)